In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [6]:
import numpy as np
import scipy as sp
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from selenium import webdriver
import datetime as dt
import time

pd.options.display.max_columns = 400
pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 600
pd.options.display.precision = 10

In [7]:
import warnings
warnings.filterwarnings("ignore")

## 특이사항

* ### 언더웨어 카테고리에 침구류가 속해있음
- 10/13 TV쇼핑 방송 상품 중 02:00 ~ 06:00 4가지 상품이 침구 상품이나 언더웨어 카테고리에 속해있음
  - 2070064444 / 2070064276 / 2070064553 / 2070064805
  
* ### 카테고리로 확인하는 경우와 날짜로 확인하는 경우의 상품의 방송 날짜가 다름
- 아래 상품 번호들은 카테고리에는 10/13 토요일 02:00 ~ 06:00로 노출되나 날짜 지정 확인에는 10/14 일요일 02:00 ~ 06:00로 노출됨
  - 2070064444 / 2070064276 / 2070064553 / 2070064805

* ### 같은 날짜일지라도 카테고리의 상품 리스트와 날짜 지정의 상품 리스트의 차이가 존재
- 아래 상품 번호는 10/12 카테고리에는 노출되지 않으나 10/12 날짜 지정에는 노출됨
  - 2068836958

In [41]:
cc=[([4.454175233840942], [0.7268481254577637], [0.4970839023590088]), 
 ([5.195505619049072], [1.1341731548309326], [0.6579272747039795]), 
 ([4.838080883026123], [1.2574429512023926], [1.0281171798706055]), 
 ([5.220234394073486], [1.0005910396575928], [0.5947654247283936]), 
 ([5.3994300365448], [0.5921938419342041], [0.5925643444061279]), 
 ([5.223188400268555], [1.051894187927246], [0.6418852806091309]),
 ([5.43953013420105], [13.936123609542847], [16.46241545677185])]

In [17]:
import numpy as np
import pandas as pd

In [42]:
np.array(cc).shape

(7, 3, 1)

In [43]:
np.array(cc).reshape(np.array(cc).shape[0], 3)

array([[ 4.45417523,  0.72684813,  0.4970839 ],
       [ 5.19550562,  1.13417315,  0.65792727],
       [ 4.83808088,  1.25744295,  1.02811718],
       [ 5.22023439,  1.00059104,  0.59476542],
       [ 5.39943004,  0.59219384,  0.59256434],
       [ 5.2231884 ,  1.05189419,  0.64188528],
       [ 5.43953013, 13.93612361, 16.46241546]])

In [24]:
cc_1 = np.array(cc).reshape(7, 3)
cc_1

array([[ 4.45417523,  0.72684813,  0.4970839 ],
       [ 5.19550562,  1.13417315,  0.65792727],
       [ 4.83808088,  1.25744295,  1.02811718],
       [ 5.22023439,  1.00059104,  0.59476542],
       [ 5.39943004,  0.59219384,  0.59256434],
       [ 5.2231884 ,  1.05189419,  0.64188528],
       [ 5.43953013, 13.93612361, 16.46241546]])

In [34]:
cc_1[0, :]

array([4.45417523, 0.72684813, 0.4970839 ])

In [36]:
cc_1.shape[1]

3

In [27]:
np.sum(cc_1[:, 0])

35.77014470100403

In [20]:
df_cc = pd.DataFrame(cc_1, columns=['time_a', 'time_b', 'time_c'])
df_cc

,time_a,time_b,time_c
0,4.454175,0.726848,0.497084
1,5.195506,1.134173,0.657927
2,4.838081,1.257443,1.028117
3,5.220234,1.000591,0.594765
4,5.399430,0.592194,0.592564
5,5.223188,1.051894,0.641885
6,5.439530,13.936124,16.462415


In [22]:
df_cc.time_a.mean()

5.110020671572004

# 1. 카테고리 지정하여 전체 크롤
* 여성의류 / 언더웨어 / 명품|잡화|제화 / 스포츠패션|아웃도어

In [8]:
def GetTVCategory():
    cate_id = {'여성의류' : 502950, '언더웨어' : 150549, '명품|잡화|제화' : 216428, '스포츠패션|아웃도어' : 807410}

    df_TV_cat  = pd.DataFrame()

    for cat in cate_id.keys():
        res = requests.get('https://www.hyundaihmall.com/front/bmc/brodPordPbdv.do?cnt=0&type=03&sectId={}&idx='.format(cate_id[cat])) 
        soup = BeautifulSoup(res.content, 'html5lib')

        num = 1
        while True:
            r_url = soup.select('#brodListTop > li:nth-of-type({}) > div > div > div > div > p > a'.format(num))
            if len(r_url) < 1:
                break
            TV_cat = pd.DataFrame()  
            url_tags, item_num = [], []    

            for url in r_url:
                url_tags.append(url)
                item_num.append(re.sub(r'slitmCd=', '', re.search(r'slitmCd=\d+', url.get('onclick'))[0]))

            TV_cat = TV_cat.assign(ITEM_NUM=item_num, CATEGORY=cat)
            df_TV_cat = df_TV_cat.append(TV_cat)

            num += 1

    df_TV_cat = df_TV_cat.reset_index(drop=True)
    df_TV_cat = df_TV_cat.drop_duplicates()
    
    return df_TV_cat

In [9]:
%%time
df_TV_cat = GetTVCategory()

Wall time: 3.05 s


In [10]:
print(df_TV_cat.shape)
df_TV_cat.head()

(248, 2)


,ITEM_NUM,CATEGORY
0,2069712779,여성의류
1,2063556558,여성의류
2,2071248579,여성의류
3,2069511042,여성의류
4,2070534904,여성의류


# 2. 이틀 뒤에 방송되는 상품들 중 1번의 ITEM_NUM과 겹치는 상품만 크롤
* 날짜를 지정하여 크롤 시 카테고리를 구분할 수 없거나 잘못된 카테고리가 나오는 경우가 많음
* 날짜 지정과 카테고리 지정의 방송시간이 다른 경우가 있는데 날짜 지정의 시간이 정확함 (Hmall 고객센터와 통화)

In [11]:
def insetHmall(df_TV_cat):
    # ITEM_NUM과 시간 block 번호, 상품 순서를 input으로 받아 1row의 data frame을 출력 하는 함수
    def GetGoodsRow(goods_num, numa, numb):
        df = pd.DataFrame()

        goods_name = [soup.select('#brodListTop > li:nth-of-type({}) > div > div > div:nth-of-type({}) > div > p > a'.format(numa, numb))[0].text.strip()]
        r_price = soup.select('#brodListTop > li:nth-of-type({}) > div > div > div:nth-of-type({}) > div > div > strong'.format(numa, numb))[0].text
        price = [re.sub(r'[^0-9]', '', r_price)]
        cat = [df_TV_cat[df_TV_cat.ITEM_NUM == goods_num].CATEGORY.values[0]]
        collect_url = ['https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=' + goods_num]

        tv_times = soup.select('#brodListTop > li:nth-of-type({}) > div > div.cell1 > p.time'.format(numa))[0].text.split(' ~ ')
        start_list = [(dt.datetime.now() + dt.timedelta(days=1)).strftime('%Y.%m.%d ') + tv_times[0]]
        end_list = [(dt.datetime.now() + dt.timedelta(days=1)).strftime('%Y.%m.%d ') + tv_times[1]]

        start_time = dt.datetime.strptime(tv_times[0], '%H:%M')
        end_time = dt.datetime.strptime(tv_times[1], '%H:%M') 
        if (end_time - start_time).total_seconds()/60 > 0:
            during = [(end_time - start_time).total_seconds()/60]
        else:
            during = [(end_time + dt.timedelta(hours=2) - (start_time + dt.timedelta(hours=2) + dt.timedelta(days=-1))).total_seconds()/60]

        df = df.assign(START=start_list, END=end_list, DURING=during, ITEM_NUM=goods_num, GOODS_NAME=goods_name, CATEGORY=cat,
                       PRICE=price, COLLECT_URL=collect_url)

        return df

    now = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # 매일의 이틀 뒤 데이터 - rquests.get에 삽입된다.
    tom_day = (dt.datetime.now() + dt.timedelta(days=1) ).strftime('%Y%m%d') 

    res = requests.get('https://www.hyundaihmall.com/front/bmc/brodPordPbdv.do?cnt=0&date={}'.format(tom_day)) 
    soup = BeautifulSoup(res.content, 'html5lib')

    # 최종 출력될 data frame
    df_tomo_TV = pd.DataFrame()

    # 카테고리 노출 ITEM의 ITEM_NUM
    num_list = np.array(df_TV_cat.ITEM_NUM)

    # 하루 총 방송 횟수
    time_tot_num = len(soup.select('#brodListTop > li > div > div.cell1 > p.time'))

    for time_num in range(1, time_tot_num+1):
        # 해당 시간에 노출되는 상품의 수
        time_goods_num = len(soup.select('#brodListTop > li:nth-of-type({}) > div > div > div > div > p > a'.format(time_num)))
        for pd_num in range(1, time_goods_num+1):

            # 상품 하나하나의 ITEM_NUM을 먼저 파악한 후 df_TV_cat 포함되어 있는 상품일 경우 crawl
            goods = soup.select('#brodListTop > li:nth-of-type({}) > div > div > div:nth-of-type({}) > div > p > a'.format(time_num, pd_num))
            goods_num = re.sub(r'slitmCd=', '', re.search(r'slitmCd=\d+', goods[0].get('onclick'))[0])
            if goods_num in num_list:
                df_tomo_TV = df_tomo_TV.append(GetGoodsRow(goods_num, time_num, pd_num))
    #             print(goods_num, df_TV_cat[df_TV_cat.ITEM_NUM == goods_num].CATEGORY.values[0], time_num, pd_num)

    df_tomo_TV = df_tomo_TV.assign(COLLECT_SITE='HMALL', REG_DT=now)
    
#     engine_dev = create_engine("mysql+pymysql://ID:PW@host/table_name?charset=utf8mb4",
#                             encoding='utf8', pool_size=20, pool_recycle=3600, connect_args={'connect_timeout': 1000000})
#     df_tomo_TV.to_sql('MWS_COLT_ITEM_TV', if_exists='append', con=engine_dev, index=False)

    return df_tomo_TV

In [12]:
%%time
df_tomo_TV = insetHmall(df_TV_cat)

Wall time: 934 ms


In [13]:
print(df_tomo_TV.shape)
df_tomo_TV.head(3)

(31, 10)


,START,END,DURING,ITEM_NUM,GOODS_NAME,CATEGORY,PRICE,COLLECT_URL,COLLECT_SITE,REG_DT
0,2018.10.27 02:10,2018.10.27 06:00,230.0,2071226379,애플라인드 남성 트리코트 허그셔츠 3종,스포츠패션|아웃도어,117000,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2071226379,HMALL,2018-10-26 11:54:35
0,2018.10.27 02:10,2018.10.27 06:00,230.0,2071227111,애플라인드 여성 트리코트 허그셔츠 3종,스포츠패션|아웃도어,117000,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2071227111,HMALL,2018-10-26 11:54:35
0,2018.10.27 02:10,2018.10.27 06:00,230.0,2069511042,라씨엔토 캐시미어100% 풀오버,여성의류,159000,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2069511042,HMALL,2018-10-26 11:54:35


# 3. 일괄실행

In [16]:
def insertColtItemTv():
    df_TV_cat = GetTVCategory()
    df_tomo_TV = insetHmall(df_TV_cat)
    return df_tomo_TV

In [17]:
insertColtItemTv()

,START,END,DURING,ITEM_NUM,GOODS_NAME,CATEGORY,PRICE,COLLECT_URL,COLLECT_SITE,REG_DT
0,2018.10.14 02:00,2018.10.14 06:00,240.0,2069469048,2018 브레라(BRERA) NEW 블레이크 고미노 로퍼 (남성용/KOMMINO),명품|잡화|제화,99000,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2069469048,HMALL,2018-10-12 10:36:26
0,2018.10.14 02:00,2018.10.14 06:00,240.0,2069468839,2018 브레라(BRERA) NEW 블레이크 고미노 로퍼 (여성용/KOMMINO),명품|잡화|제화,99000,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2069468839,HMALL,2018-10-12 10:36:26
0,2018.10.14 02:00,2018.10.14 06:00,240.0,2064464217,시네쿠아논 프리미엄 컬러 라쿤퍼 구스 야상 점퍼,여성의류,94050,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2064464217,HMALL,2018-10-12 10:36:26
0,2018.10.14 02:00,2018.10.14 06:00,240.0,2070064444,네이처썸 링클 피그먼트 패드 1+1 세트 (Q),언더웨어,89910,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2070064444,HMALL,2018-10-12 10:36:26
0,2018.10.14 02:00,2018.10.14 06:00,240.0,2070064276,네이처썸 링클 피그먼트 패드 1+1 세트 (SS),언더웨어,80910,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2070064276,HMALL,2018-10-12 10:36:26
0,2018.10.14 02:00,2018.10.14 06:00,240.0,2070064553,네이처썸 링클 피그먼트 패드 1+1 세트 (K),언더웨어,107910,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2070064553,HMALL,2018-10-12 10:36:26
0,2018.10.14 02:00,2018.10.14 06:00,240.0,2070064805,네이처썸 링클 피그먼트 패드 1+1 세트 (SK),언더웨어,116910,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2070064805,HMALL,2018-10-12 10:36:26
0,2018.10.14 02:00,2018.10.14 06:00,240.0,2070214855,벤호건 남성 핫슬러에디션 6종,스포츠패션|아웃도어,79000,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2070214855,HMALL,2018-10-12 10:36:26
0,2018.10.14 02:00,2018.10.14 06:00,240.0,2070215064,벤호건 여성 핫슬러에디션 6종,스포츠패션|아웃도어,79000,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2070215064,HMALL,2018-10-12 10:36:26


In [15]:
def insertExecutePlan(siteName,df_TV):
    # create 7 time points	
    def create_7points(siteName,row):
        start = pd.to_datetime(row['START']) # to_datetime
        temp_ = (start + dt.timedelta(hours=+35))#+dt.timedelta(days=1) # "replace" would make error if the day is the last of the month (10.10)
        act_df_list = {'ACT_DT' : [start + dt.timedelta(days=-1),
                                   start + dt.timedelta(hours=-1),
                                   start + dt.timedelta(hours=3),
                                   start + dt.timedelta(hours=7),
                                   start + dt.timedelta(hours=11),
                                   temp_.replace(hour=0, minute=1),
                                   temp_.replace(hour=4, minute=1)
                                   ]}  # 2nd_crawler의 중복 시점 방지를 위해 정각이 아닌 1분 추가
        act_df = pd.DataFrame(act_df_list)
        return act_df.assign(COLLECT_SITE=siteName, ITEM_NUM = str(row['ITEM_NUM']), ACT_DT = act_df.ACT_DT.astype(str)) 

    # 기존 db 에 존재하는 item_num 제외하기
    def filterDuplicates_ExecutePlan(siteName,df_new,engine_os):                
        df_db = 0 
        try:
            query_url = """
            select ITEM_NUM, ACT_DT from MWS_COLT_ITEM_EXECUTE_PLAN
            where COLLECT_SITE = '{}'
            and ITEM_NUM in {} 
            """
            df_db = pd.read_sql_query(query_url.format(siteName,tuple(df_new.ITEM_NUM.unique())), engine_os)
        except:
            query_url = """
            select ITEM_NUM, ACT_DT from MWS_COLT_ITEM_EXECUTE_PLAN
            where COLLECT_SITE = '{}'
            and ITEM_NUM in ('{}')
            """ # item_num이 1개인 경우, 에러방지위해 string으로 넣는다.
            df_db = pd.read_sql_query(      query_url.format(siteName,"','".join(df_new.ITEM_NUM.unique())  ), engine_os)        
        not_duple_df = df_new[~(df_new.ITEM_NUM.isin(df_db.ITEM_NUM.unique()) & df_new.ACT_DT.isin(df_db.ACT_DT.unique()))]
        not_duple_df['REG_DT'] = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        return not_duple_df
    
    # 다음 두개 함수에서 연속에서 사용할 connection 
    engine_os = create_engine("mysql+pymysql://ID:PW@host/table_name?charset=utf8mb4",
                              encoding='utf8', pool_size=20, pool_recycle=3600,
                              connect_args={'connect_timeout': 1000000})

    df_execute_plan = pd.concat([create_7points(siteName,item) for idx, item in df_TV.iterrows()])
    df_execute_plan.drop_duplicates(inplace=True)
    df_execute_plan_new = filterDuplicates_ExecutePlan(siteName,df_execute_plan, engine_os)
#     if len(df_execute_plan_new)>0:
#         print(df_execute_plan_new.shape,df_execute_plan_new)
#     df_execute_plan_new.to_sql('MWS_COLT_ITEM_EXECUTE_PLAN', if_exists='append', con=engine_os, index=False)
    return df_execute_plan_new

In [16]:
df_execute = insertExecutePlan('www.hyundaihmall.com',df_tomo_TV)

In [18]:
print(df_execute.shape)
df_execute.head()

(217, 4)


,ACT_DT,COLLECT_SITE,ITEM_NUM,REG_DT
0,2018-10-26 02:10:00,www.hyundaihmall.com,2071226379,2018-10-26 11:55:06
1,2018-10-27 01:10:00,www.hyundaihmall.com,2071226379,2018-10-26 11:55:06
2,2018-10-27 05:10:00,www.hyundaihmall.com,2071226379,2018-10-26 11:55:06
3,2018-10-27 09:10:00,www.hyundaihmall.com,2071226379,2018-10-26 11:55:06
4,2018-10-27 13:10:00,www.hyundaihmall.com,2071226379,2018-10-26 11:55:06


In [28]:
engine_os = create_engine("mysql+pymysql://ID:PW@host/table_name?charset=utf8mb4",
                          encoding='utf8', pool_size=20, pool_recycle=3600,
                          connect_args={'connect_timeout': 1000000})

In [36]:
df_execute.to_sql('MWS_COLT_ITEM_EXECUTE_PLAN', if_exists='append', con=engine_os, index=False)

In [37]:
df_execute.shape
df_execute.head()

,ACT_DT,COLLECT_SITE,ITEM_NUM,REG_DT
0,2018-10-13 02:00:00,www.hyundaimall.com,2069469048,2018-10-12 11:16:21
1,2018-10-14 01:00:00,www.hyundaimall.com,2069469048,2018-10-12 11:16:21
2,2018-10-14 05:00:00,www.hyundaimall.com,2069469048,2018-10-12 11:16:21
3,2018-10-14 09:00:00,www.hyundaimall.com,2069469048,2018-10-12 11:16:21
4,2018-10-14 13:00:00,www.hyundaimall.com,2069469048,2018-10-12 11:16:21


In [45]:
query = """
select *
from MWS_COLT_ITEM_EXECUTE_PLAN
where collect_site = 'www.hyundaimall.com'
and reg_dt > '2018-10-12 11:16:00'
limit 100
"""

In [46]:
pd.read_sql_query(query, engine_os).shape

(63, 5)

In [ ]:
# delete from MWS_COLT_ITEM_EXECUTE_PLAN where collect_site='www.hyundaimall.com' and reg_dt > '2018-10-12 11:16:00';

In [ ]:
delete from MWS_COLT_ITEM_EXECUTE_PLAN where reg_dt>'2018-10-12 오후 3:08:50'

In [16]:
# 1. 카테고리 지정하여 전체 크롤 ========================================================================================================
def GetTVCategory():
    cate_id = {'여성의류' : 502950, '언더웨어' : 150549, '명품|잡화|제화' : 216428, '스포츠패션|아웃도어' : 807410}

    df_TV_cat  = pd.DataFrame()

    for cat in cate_id.keys():
        res = requests.get('https://www.hyundaihmall.com/front/bmc/brodPordPbdv.do?cnt=0&type=03&sectId={}&idx='.format(cate_id[cat])) 
        soup = BeautifulSoup(res.content, 'html5lib')

        num = 1
        while True:
            r_url = soup.select('#brodListTop > li:nth-of-type({}) > div > div > div > div > p > a'.format(num))
            if len(r_url) < 1:
                break
            TV_cat = pd.DataFrame()  
            url_tags, item_num = [], []    

            for url in r_url:
                url_tags.append(url)
                item_num.append(re.sub(r'slitmCd=', '', re.search(r'slitmCd=\d+', url.get('onclick'))[0]))

            TV_cat = TV_cat.assign(ITEM_NUM=item_num, CATEGORY=cat)
            df_TV_cat = df_TV_cat.append(TV_cat)

            num += 1

    df_TV_cat = df_TV_cat.reset_index(drop=True)
    df_TV_cat = df_TV_cat.drop_duplicates()
    
    return df_TV_cat



# 2. 이틀 뒤에 방송되는 상품들 중 1번의 ITEM_NUM과 겹치는 상품만 크롤 ================================================================
def insetHmall(df_TV_cat):
    # ITEM_NUM과 시간 block 번호, 상품 순서를 input으로 받아 1row의 data frame을 출력 하는 함수
    def GetGoodsRow(goods_num, numa, numb):
        df = pd.DataFrame()

        goods_name = [soup.select('#brodListTop > li:nth-of-type({}) > div > div > div:nth-of-type({}) > div > p > a'.format(numa, numb))[0].text.strip()]
        r_price = soup.select('#brodListTop > li:nth-of-type({}) > div > div > div:nth-of-type({}) > div > div > strong'.format(numa, numb))[0].text
        price = [re.sub(r'[^0-9]', '', r_price)]
        cat = [df_TV_cat[df_TV_cat.ITEM_NUM == goods_num].CATEGORY.values[0]]
        collect_url = ['https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=' + goods_num]

        tv_times = soup.select('#brodListTop > li:nth-of-type({}) > div > div.cell1 > p.time'.format(numa))[0].text.split(' ~ ')
        start_list = [(dt.datetime.now() + dt.timedelta(days=2)).strftime('%Y.%m.%d ') + tv_times[0]]
        end_list = [(dt.datetime.now() + dt.timedelta(days=2)).strftime('%Y.%m.%d ') + tv_times[1]]

        start_time = dt.datetime.strptime(tv_times[0], '%H:%M')
        end_time = dt.datetime.strptime(tv_times[1], '%H:%M') 
        if (end_time - start_time).total_seconds()/60 > 0:
            during = [(end_time - start_time).total_seconds()/60]
        else:
            during = [(end_time + dt.timedelta(hours=2) - (start_time + dt.timedelta(hours=2) + dt.timedelta(days=-1))).total_seconds()/60]

        df = df.assign(START=start_list, END=end_list, DURING=during, ITEM_NUM=goods_num, GOODS_NAME=goods_name, CATEGORY=cat,
                       PRICE=price, COLLECT_URL=collect_url)

        return df

    now = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # 매일의 이틀 뒤 데이터 - rquests.get에 삽입된다.
    tom_day = (dt.datetime.now() + dt.timedelta(days=2)).strftime('%Y%m%d')

    res = requests.get('https://www.hyundaihmall.com/front/bmc/brodPordPbdv.do?cnt=0&date={}'.format(tom_day)) 
    soup = BeautifulSoup(res.content, 'html5lib')

    # 최종 출력될 data frame
    df_tomo_TV = pd.DataFrame()

    # 카테고리 노출 ITEM의 ITEM_NUM
    num_list = np.array(df_TV_cat.ITEM_NUM)

    # 하루 총 방송 횟수
    time_tot_num = len(soup.select('#brodListTop > li > div > div.cell1 > p.time'))

    for time_num in range(1, time_tot_num+1):
        # 해당 시간에 노출되는 상품의 수
        time_goods_num = len(soup.select('#brodListTop > li:nth-of-type({}) > div > div > div > div > p > a'.format(time_num)))
        for pd_num in range(1, time_goods_num+1):

            # 상품 하나하나의 ITEM_NUM을 먼저 파악한 후 df_TV_cat 포함되어 있는 상품일 경우 crawl
            goods = soup.select('#brodListTop > li:nth-of-type({}) > div > div > div:nth-of-type({}) > div > p > a'.format(time_num, pd_num))
            goods_num = re.sub(r'slitmCd=', '', re.search(r'slitmCd=\d+', goods[0].get('onclick'))[0])
            if goods_num in num_list:
                df_tomo_TV = df_tomo_TV.append(GetGoodsRow(goods_num, time_num, pd_num))
    #             print(goods_num, df_TV_cat[df_TV_cat.ITEM_NUM == goods_num].CATEGORY.values[0], time_num, pd_num)

    df_tomo_TV = df_tomo_TV.assign(COLLECT_SITE='HMALL', REG_DT=now)
    
#     engine_dev = create_engine("mysql+pymysql://ID:PW@host/table_name", 
#                                pool_size=20, pool_recycle=3600, connect_args={'connect_timeout': 1000000})
#     df_tomo_TV.to_sql('MWS_COLT_ITEM_TV', if_exists='append', con=engine_dev, index=False)

    return df_tomo_TV


# 3. 일괄 실행 ====================================================================================================================
def insertColtItemTv():
    df_TV_cat = GetTVCategory()
    df_tomo_TV = insetHmall(df_TV_cat)
    return df_tomo_TV

In [18]:
%%time
aa = insertColtItemTv()

Wall time: 2.32 s


# src

In [ ]:
# 1. 카테고리 지정하여 전체 크롤 ========================================================================================================
def GetTVCategory():
    cate_id = {'여성의류' : 502950, '언더웨어' : 150549, '명품|잡화|제화' : 216428, '스포츠패션|아웃도어' : 807410}

    df_TV_cat  = pd.DataFrame()

    for cat in cate_id.keys():
        res = requests.get('https://www.hyundaihmall.com/front/bmc/brodPordPbdv.do?cnt=0&type=03&sectId={}&idx='.format(cate_id[cat])) 
        soup = BeautifulSoup(res.content, 'html5lib')

        num = 1
        while True:
            r_url = soup.select('#brodListTop > li:nth-of-type({}) > div > div > div > div > p > a'.format(num))
            if len(r_url) < 1:
                break
            TV_cat = pd.DataFrame()  
            url_tags, item_num = [], []    

            for url in r_url:
                url_tags.append(url)
                item_num.append(re.sub(r'slitmCd=', '', re.search(r'slitmCd=\d+', url.get('onclick'))[0]))

            TV_cat = TV_cat.assign(ITEM_NUM=item_num, CATEGORY=cat)
            df_TV_cat = df_TV_cat.append(TV_cat)

            num += 1

    df_TV_cat = df_TV_cat.reset_index(drop=True)
    df_TV_cat = df_TV_cat.drop_duplicates()
    
    return df_TV_cat



# 2. 이틀 뒤에 방송되는 상품들 중 1번의 ITEM_NUM과 겹치는 상품만 크롤 ================================================================
def insetHmall(df_TV_cat):
    # ITEM_NUM과 시간 block 번호, 상품 순서를 input으로 받아 1row의 data frame을 출력 하는 함수
    def GetGoodsRow(goods_num, numa, numb):
        df = pd.DataFrame()

        goods_name = [soup.select('#brodListTop > li:nth-of-type({}) > div > div > div:nth-of-type({}) > div > p > a'.format(numa, numb))[0].text.strip()]
        r_price = soup.select('#brodListTop > li:nth-of-type({}) > div > div > div:nth-of-type({}) > div > div > strong'.format(numa, numb))[0].text
        price = [re.sub(r'[^0-9]', '', r_price)]
        cat = [df_TV_cat[df_TV_cat.ITEM_NUM == goods_num].CATEGORY.values[0]]
        collect_url = ['https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=' + goods_num]

        tv_times = soup.select('#brodListTop > li:nth-of-type({}) > div > div.cell1 > p.time'.format(numa))[0].text.split(' ~ ')
        start_list = [(dt.datetime.now() + dt.timedelta(days=2)).strftime('%Y.%m.%d ') + tv_times[0]]
        end_list = [(dt.datetime.now() + dt.timedelta(days=2)).strftime('%Y.%m.%d ') + tv_times[1]]

        start_time = dt.datetime.strptime(tv_times[0], '%H:%M')
        end_time = dt.datetime.strptime(tv_times[1], '%H:%M') 
        if (end_time - start_time).total_seconds()/60 > 0:
            during = [(end_time - start_time).total_seconds()/60]
        else:
            during = [(end_time + dt.timedelta(hours=2) - (start_time + dt.timedelta(hours=2) + dt.timedelta(days=-1))).total_seconds()/60]

        df = df.assign(START=start_list, END=end_list, DURING=during, ITEM_NUM=goods_num, GOODS_NAME=goods_name, CATEGORY=cat,
                       PRICE=price, COLLECT_URL=collect_url)

        return df

    now = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # 매일의 이틀 뒤 데이터 - rquests.get에 삽입된다.
    tom_day = (dt.datetime.now() + dt.timedelta(days=2)).strftime('%Y%m%d')

    res = requests.get('https://www.hyundaihmall.com/front/bmc/brodPordPbdv.do?cnt=0&date={}'.format(tom_day)) 
    soup = BeautifulSoup(res.content, 'html5lib')

    # 최종 출력될 data frame
    df_tomo_TV = pd.DataFrame()

    # 카테고리 노출 ITEM의 ITEM_NUM
    num_list = np.array(df_TV_cat.ITEM_NUM)

    # 하루 총 방송 횟수
    time_tot_num = len(soup.select('#brodListTop > li > div > div.cell1 > p.time'))

    for time_num in range(1, time_tot_num+1):
        # 해당 시간에 노출되는 상품의 수
        time_goods_num = len(soup.select('#brodListTop > li:nth-of-type({}) > div > div > div > div > p > a'.format(time_num)))
        for pd_num in range(1, time_goods_num+1):

            # 상품 하나하나의 ITEM_NUM을 먼저 파악한 후 df_TV_cat 포함되어 있는 상품일 경우 crawl
            goods = soup.select('#brodListTop > li:nth-of-type({}) > div > div > div:nth-of-type({}) > div > p > a'.format(time_num, pd_num))
            goods_num = re.sub(r'slitmCd=', '', re.search(r'slitmCd=\d+', goods[0].get('onclick'))[0])
            if goods_num in num_list:
                df_tomo_TV = df_tomo_TV.append(GetGoodsRow(goods_num, time_num, pd_num))
    #             print(goods_num, df_TV_cat[df_TV_cat.ITEM_NUM == goods_num].CATEGORY.values[0], time_num, pd_num)

    df_tomo_TV = df_tomo_TV.assign(COLLECT_SITE='HMALL', REG_DT=now)
    
#     engine_dev = create_engine("mysql+pymysql://ID:PW@host/table_name?charset=utf8mb4",
#                             encoding='utf8', pool_size=20, pool_recycle=3600, connect_args={'connect_timeout': 1000000})
#     df_tomo_TV.to_sql('MWS_COLT_ITEM_TV', if_exists='append', con=engine_dev, index=False)

    return df_tomo_TV


# 3. 일괄 실행 ====================================================================================================================
def insertColtItemTv():
    df_TV_cat = GetTVCategory()
    df_tomo_TV = insetHmall(df_TV_cat)
#     return df_tomo_TV

# dag

In [ ]:
from src.src_Hmall_TV_shopping_crawler import * # python file import
from airflow.models import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.operators.bash_operator import BashOperator

from datetime import datetime, timedelta

default_args = {
    'owner' : 'ben', # airflow에 표시되는 사용자명
    'depends_on_past':False, # 이전 task에 이어서 작업하는 지 여부
    'start_date':datetime(2018,10,11), # 시작 날짜
    'retries':1, # 실패가 되기 전 재시도 횟수
    'retry_delay':timedelta(minutes=1) # 재시도 시간
}

#airflow job에는 TV_2nd_Crawler이 올라간다
dag = DAG('Hmall_TV_shopping_crawler',
         default_args = default_args,
         dagrun_timeout=timedelta(days=3), # 실패 전까지 DagRun 실행 시간 
         description="Crawler_Hmall_TV_shopping", # DAG에 대한 설명
         schedule_interval='0 23 * * *') # DAG의 실행 빈도
                    # at 23 oclock, it works . but it does not show the right time number on time
                    # and I guess that it is only working on 133.186.146.49 server 
                    # it takes about 2min to adjust a new code to online code

crawl_operator = PythonOperator( 
        task_id='hmall_tv_shopping_crawler',
        python_callable=insertColtItemTv, # 호출 함수 
        queue='q_development',
        dag=dag)

waiting_operator = BashOperator(
        task_id = 'sleep',
        bash_command='sleep 1',
        queue='q_development',
        dag=dag)

waiting_operator >> crawl_operator


# MWS_COLT_ITEM_TV hmall 데이터 수정
* ITEM_NUM의 길이가 GSShop은 8자리, Hmall은 10자리
* 초기 MWS_COLT_ITEM_TV 테이블 생성 당시, GSShop만을 고려하여 column 속성을 8자리로 고정해놓았음
* 때문에 column 속성의 문자 길이 제한으로 인해 뒤 2자리가 생략되어 입력됨
* 테이블 내의 모든 Hmall 데이터를 불러 온전한 ITEM_NUM으로 수정

In [210]:
engine_dev = create_engine("mysql+pymysql://ID:PW@host/table_name", 
                           pool_size=20, pool_recycle=3600, connect_args={'connect_timeout': 1000000})

In [230]:
query = """
select * 
from MWS_COLT_ITEM_TV
where COLLECT_SITE = 'HMALL'
"""

In [231]:
hmall_tv = pd.read_sql_query(query, engine_dev)

* COLLECT_URL column을 통하여 올바른 ITEM_NUM이 입력된 테이블 생성

In [176]:
modi_item_num = [re.sub(r'slitmCd=', '', re.search(r'slitmCd=\d+', hamll_url)[0]) for hamll_url in hmall_tv.COLLECT_URL]
hmall_tv.ITEM_NUM = modi_item_num

* db 각 row id에 맞게 수정된 ITEM_NUM 입력

In [225]:
hmall_id = hmall_tv.ID.tolist()

In [229]:
%%time
for i in range(len(hmall_tv)):
    query = """
    UPDATE MWS_COLT_ITEM_TV SET ITEM_NUM = '{}' WHERE ID = {}
    """.format(modi_item_num[i], hmall_id[i])
    engine_dev.execute(query)

Wall time: 744 ms


* 수정된 값 확인

In [233]:
query = """
select * 
from MWS_COLT_ITEM_TV
where COLLECT_SITE = 'HMALL'
"""

In [234]:
hmall_tv = pd.read_sql_query(query, engine_dev)

In [235]:
hmall_tv[['ID', 'ITEM_NUM', 'COLLECT_URL']]

,ID,ITEM_NUM,COLLECT_URL
0,636,2069469048,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2069469048
1,637,2069468839,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2069468839
2,638,2064464217,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2064464217
3,639,2070064444,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2070064444
4,640,2070064276,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2070064276
5,641,2070064553,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2070064553
6,642,2070064805,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2070064805
7,643,2070214855,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2070214855
8,644,2070215064,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2070215064
9,645,2069616462,https://www.hyundaihmall.com/front/pda/itemPtc.do?slitmCd=2069616462


In [ ]:
hmall_tv

In [200]:
type(hmall_tv.ID[0])

numpy.int64

In [236]:
engine_os = create_engine("mysql+pymysql://ID:PW@host/table_name?charset=utf8mb4", encoding = 'utf8',
                       pool_size=20,pool_recycle=3600,connect_args={'connect_timeout':1000000} )

In [ ]:
query = """
select * 
from MWS_COLT_ITEM_TV
where COLLECT_SITE = 'HMALL'
"""